In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
from tqdm import tqdm

In [2]:
def calc_prob(channels, ydim, xdim, freqID, src_ID, noise, noise_level):
    
    ref = 'BON'
    
    #Pre-allocate array with probabilities
    probs_tw = np.zeros((xdim*ydim))
    
    # Energy ratios are calculated for each component in respect to the chosen
    # reference station
    for ch in channels:
        
        # Energy ratios from simulated signal
        if ch == 'Z':
            direc_comp = 'data/simu/vertZ/'
        elif ch == 'E':
            direc_comp = 'data/simu/horzE/'
        elif ch == 'N':
            direc_comp = 'data/simu/horzN/'
            
            
        if ch == 'Z':
            stas_name = ['BOR', 'DSO', 'SNE']
        else:
            stas_name = ['BOR', 'SNE']

        nRatios = len(stas_name)
        ratio_obs = np.zeros(nRatios)   
        
        path_simu_ref = direc_comp + ref + '/' 
        energy_simu_ref = np.loadtxt(path_simu_ref + freqID + '/energy_Fz.txt')
        #observed energy
        energy_curr_ref  = energy_simu_ref[src_ID]
        if noise:
            energy_curr_ref += np.abs(np.random.normal(0,noise_level))
        #simulated energy catalogue
        ###energy_simu_ref = np.reshape(energy_simu_ref,(ydim,xdim))
        ratios_simu = np.zeros((ydim*xdim,nRatios)) 
        
        for idxSrc, sta in enumerate(stas_name):
            path_simu = direc_comp  + sta + '/' 
            energy_simu = np.loadtxt(path_simu + freqID + '/energy_Fz.txt')
            #observed energy
            energy_curr = energy_simu[src_ID] 
            if noise:
                energy_curr += np.abs(np.random.normal(0,noise_level))
            ratio_obs[idxSrc] = energy_curr/energy_curr_ref
            #simulated energy catalogue
            ###energy_simu = np.reshape(energy_simu,(ydim,xdim))
            ratios_simu[:,idxSrc] = energy_simu/energy_simu_ref
                    
        # Error between observed and simulated energy ratios
        for ii in range(xdim*ydim):
            for kk in range(nRatios):
                probs_tw[ii] += 1./nRatios * (
                    np.abs(np.log10(ratios_simu[ii,kk]/ratio_obs[kk])))

    # Probability is defined as inverse of error, normalized by 
    # number of channels
    probs_tw = 1./(probs_tw/len(channels))

    return(probs_tw)

In [3]:
def exponential(x, a, k, b):
    return a*np.exp(x*k) + b

In [10]:
noise = False

channels = ['E', 'N', 'Z']
freqID   = 'BP8to12'

# noise level
if freqID == 'BP3to7':
    noise_level = 1.025e-24
elif freqID == 'BP8to12':
    noise_level = 1.050e-24
elif freqID == 'BP13to17':
    noise_level = 3.277e-26
    
    
xdim = 121
ydim = 101

## Load topography
topo   = np.loadtxt('data/DEM_PF_10m_cut.dat',skiprows=6)
dx = 10
dy = 10
xx = np.arange(0.,2100.+dx,dx)
yy = np.arange(0.,1800.+dy,dy)
levels = np.arange(2190., 2650, 20)
### Load stations
srcs   = np.loadtxt('data/stations.txt')
srcs_array = ['BON', 'BOR', 'DSO', 'SNE']
xmin =  640.
xmax = 1840.
ymin =  400.
ymax = 1400.

# Loading grid positions
data = pd.read_csv('data/positions.txt', sep='\s+', 
                   header=None, names=['x', 'y', 'z']) 

In [11]:
data

,x,y,z
0,640.0,400.0,671.8667
1,650.0,400.0,672.2667
2,660.0,400.0,672.6667
3,670.0,400.0,673.0667
4,680.0,400.0,673.8667
...,...,...,...
12216,1800.0,1400.0,611.2667
12217,1810.0,1400.0,606.8667
12218,1820.0,1400.0,603.2667
12219,1830.0,1400.0,598.8667


In [12]:
#resolution = np.zeros((xdim*ydim))
resolution_popt = np.zeros((xdim*ydim, 3))
resolution_perr = np.zeros((xdim*ydim, 3))

for index, row in tqdm(data.iterrows()):
    
    radius = np.array(np.sqrt((data.x-row.x)**2 + (data.y-row.y)**2))
    probs  = calc_prob(channels, ydim, xdim, freqID, index, noise, noise_level)
    
    radius = np.delete(radius, index)
    probs  = np.delete(probs,  index)
    
    try:
        popt_exponential, pcov_exponential = optimize.curve_fit(exponential, radius, probs, p0=[10, -0.5, 0])
        perr_exponential = np.sqrt(np.diag(pcov_exponential))
        
        resolution_popt[index,:] = popt_exponential
        resolution_perr[index,:] = perr_exponential
        
        #y0 = exponential(0., popt_exponential[0], popt_exponential[1], popt_exponential[2])
        #xhalf = np.log((y0/2 - popt_exponential[2])/popt_exponential[0])/popt_exponential[1]
    except RuntimeError:
        print('Fit not found:')
        print(index)
        resolution_popt[index,:] = [np.nan, np.nan, np.nan]
        resolution_perr[index,:] = [np.nan, np.nan, np.nan]

0it [00:00, ?it/s]/home/julian/Programs/anaconda2/envs/obspy/lib/python3.7/site-packages/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in true_divide
/home/julian/Programs/anaconda2/envs/obspy/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/home/julian/Programs/anaconda2/envs/obspy/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in multiply
  
1788it [21:47,  1.33it/s]

Fit not found:
1787


1789it [21:48,  1.28it/s]

Fit not found:
1788


1790it [21:49,  1.26it/s]

...

3710it [44:37,  1.02it/s]

Fit not found:
3709


3788it [45:51,  1.17it/s]

Fit not found:
3787


12221it [2:21:41,  1.44it/s]


In [13]:
#data['res'] = resolution
data['res_a']     = resolution_popt[:,0]
data['res_a_err'] = resolution_perr[:,0]
data['res_k']     = resolution_popt[:,1]
data['res_k_err'] = resolution_perr[:,1]
data['res_b']     = resolution_popt[:,2]
data['res_b_err'] = resolution_perr[:,2]

In [14]:
data.head()

,x,y,z,res_a,res_a_err,res_k,res_k_err,res_b,res_b_err
0,640.0,400.0,671.8667,4.798513,0.037433,-0.005156,0.000041,0.518666,0.003398
1,650.0,400.0,672.2667,5.996956,0.038499,-0.005778,0.000037,0.513021,0.003105
2,660.0,400.0,672.6667,6.623006,0.037588,-0.005965,0.000035,0.493649,0.003041
3,670.0,400.0,673.0667,9.024214,0.056944,-0.007457,0.000047,0.529177,0.003605
4,680.0,400.0,673.8667,8.375678,0.051853,-0.007118,0.000045,0.523604,0.003584


In [15]:
data.to_csv('resolution/resolution_exp_ENZ_'+freqID+'.csv', index=False)